In [ ]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData
from pyspark.sql.functions import col,explode

In [22]:
stockdata = GetStockData() 

json_data  = stockdata.get_historical_stock_data()

display(json_data)

data saved for symbol QQQ and date range 2021-01-01 to 2021-12-31.
data saved for symbol VOO and date range 2021-01-01 to 2021-12-31.


[{'meta': {'symbol': 'QQQ',
   'interval': '1day',
   'currency': 'USD',
   'exchange_timezone': 'America/New_York',
   'exchange': 'NASDAQ',
   'mic_code': 'XNMS',
   'type': 'ETF'},
  'values': [{'datetime': '2021-12-30',
    'open': '401.32999',
    'high': '403.57001',
    'low': '399.75',
    'close': '400.35001',
    'volume': '23850100'},
   {'datetime': '2021-12-29',
    'open': '401.75',
    'high': '403.049988',
    'low': '399.10999',
    'close': '401.54999',
    'volume': '32248500'},
   {'datetime': '2021-12-28',
    'open': '404.39999',
    'high': '404.57999',
    'low': '400.70999',
    'close': '401.60999',
    'volume': '33113700'},
   {'datetime': '2021-12-27',
    'open': '398.25',
    'high': '403.48001',
    'low': '398.22000',
    'close': '403.48001',
    'volume': '32820700'},
   {'datetime': '2021-12-23',
    'open': '394.31000',
    'high': '398.42999',
    'low': '394.14001',
    'close': '396.92001',
    'volume': '29595500'},
   {'datetime': '2021-12-22',

In [24]:
df = spark.createDataFrame(json_data)  
df_flat = df.select( df.meta.symbol.alias("symbol"),"values")
display(df_flat.show()) 

+------+--------------------+
|symbol|              values|
+------+--------------------+
|   QQQ|[{volume -> 23850...|
|   VOO|[{volume -> 37518...|
+------+--------------------+



None

In [64]:
df_flat2 =    df.selectExpr(
        "meta['symbol'] as symbol",
        "values"
)

df_flat_exploded = df_flat2.select(
                        col("symbol"),
                        explode( "values").alias("record"))
                        
                        # .withColumn( "volume" , col("record").getItem("volume") ) \
                        # .withColumn( "high" , col("record").getItem("high") ) \
                        # .withColumn( "low" , col("record").getItem("low") ) \
                        # .withColumn( "close" , col("record").getItem("close") ) \
                        # .withColumn( "open" , col("record").getItem("open") ) \
                        # .withColumn( "date" , col("record").getItem("datetime") ) \
                        #         .drop("record")


df_flat_exploded =   df_flat_exploded.select(
        "symbol",
        col("record").getItem("volume").alias("volume"),
        col("record").getItem("high").alias("high"),
        col("record").getItem("low").alias("low"),
        col("record").getItem("close").alias("close"),
        col("record").getItem("open").alias("open"),
        col("record").getItem("date").alias("date")
        
  )   
df_flat_exploded = df_flat_exploded.filter(  col("symbol")  == "VOO" ).distinct().count()     


print(df_flat_exploded)
# df_flat_exploded.show(truncate = False )

251
